<a href="https://colab.research.google.com/github/tsycstang/DL_practices/blob/master/text_generation/textgenrnn_weibo_hu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
data_raw = pd.read_csv('/content/1989660417.csv')

In [0]:
data_raw['正文'].to_csv('weibos.csv', index=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [0]:
!pip install textgenrnn
from textgenrnn import textgenrnn
from datetime import datetime
import os

Using TensorFlow backend.


In [0]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 30,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 1.0,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [0]:
file_name = "weibos.csv"
model_name = 'textgenrnn-weibo-hu'   # change to set file name of resulting trained models/texts

In [0]:
#textgen = textgenrnn(name=model_name)

#train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=False,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

5,154 texts collected.
Training on 1,022,304 character sequences.
Epoch 1/30
998/998 [==============================] - 237s 237ms/step - loss: 2.5210
Epoch 2/30
998/998 [==============================] - 236s 237ms/step - loss: 2.5229
Epoch 3/30
998/998 [==============================] - 238s 238ms/step - loss: 2.4811
Epoch 4/30
998/998 [==============================] - 239s 239ms/step - loss: 2.4330
Epoch 5/30
998/998 [==============================] - 237s 238ms/step - loss: 2.3828
####################
Temperature: 0.2
####################
#胡侃# 【美国再次向中国提供核科学研项工，但是中国的一部方法，也是美国的一个方向。中国的对美开展得到充满思想和变化，中国的实力和智慧和顺畅，也是中国的强大阻力。中国的国力和量力有着复杂的成就，这是我们的经济成长本身，我们的社会就会有更多的成功。 中国的人口总量有，中国的崛起是最大的国家，也是我们的巨大市场，这是我们必须有的效果。                                                                                                                                    

我在微访谈"胡锡进谈国家“边疆节”"中回答了网友 @秋飞小龙  的提问, 

#胡侃# 【美国又要花钱就抹黑中国的能力？中国不能不相信美国，美国的自我优先选项比起来，我们的民生就是美国的强大，也是中国的命运最大限地。 

####################
Temperature: 0.5
#

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
#files.download(gen_file)